# Projeto: *Music Generation*
* Geração da próxima nota musical.
* Neste exemplo, utilizaremos alguns dos *Chorais de Bach* para gerar músicas. Os chorais são uma parte importante da música sacra de Bach e são frequentemente encontrados em suas obras corais, como as cantatas, os oratórios e a Paixão de São Mateus. Os chorais de Bach são caracterizados por suas melodias ricas e harmonias sofisticadas, e muitos deles se tornaram amplamente conhecidos e apreciados até hoje.

Para essa aplicação, utilizaremos um [conjunto de dados](https://drive.google.com/drive/folders/1dKOdOhmcijZhoZEYwDJ-to3O4jAfiiD_?usp=sharing) contendo alguns dos Corais de Bach.

Este conjunto é composto por 382 corais.

Cada coral tem entre 100 e 640 passos de tempo e cada passo de tempo contém 4 inteiros, onde cada inteiro corresponde ao índice de uma nota em um piano (exceto pelo valor 0, que significa que nenhuma nota é tocada). 

Treine um modelo - recorrente, convolucional ou ambos - que possa prever o próximo passo de tempo (quatro notas), dado uma sequência de passos de tempo de um coral. 

Em seguida, use esse modelo para gerar música semelhante a Bach, uma nota de cada vez: você pode fazer isso fornecendo ao modelo o início de um coral e pedindo para ele prever o próximo passo de tempo, em seguida, acrescentando esses passos de tempo à sequência de entrada e pedindo ao modelo pela próxima nota, e assim por diante. 

## Setup

In [1]:
# Python ≥3.5 - requerido
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Imports comuns
import numpy as np
import os
from pathlib import Path

# Estabilizar e padronizar as respostas deste notebook.
np.random.seed(42)
tf.random.set_seed(42)

## Dataset contendo os chorais de Bach

In [2]:
# Acessando e baixando os arquivos
DOWNLOAD_ROOT = "https://tinyurl.com/2d2ammnn/"
FILENAME = "jsb_chorales.tgz"
filepath = keras.utils.get_file(FILENAME,
                                DOWNLOAD_ROOT + FILENAME,
                                cache_subdir="datasets/jsb_chorales",
                                extract=True)

117661/117661 [==============================] - 0s 0us/step


In [3]:
# Obtém o diretório pai do caminho de arquivo especificado
jsb_chorales_dir = Path(filepath).parent

# Encontra todos os arquivos que correspondem ao padrão "train/chorale_*.csv" no diretório
train_files = sorted(jsb_chorales_dir.glob("train/chorale_*.csv"))

# Encontra todos os arquivos que correspondem ao padrão "valid/chorale_*.csv" no diretório
valid_files = sorted(jsb_chorales_dir.glob("valid/chorale_*.csv"))

# Encontra todos os arquivos que correspondem ao padrão "test/chorale_*.csv" no diretório
test_files = sorted(jsb_chorales_dir.glob("test/chorale_*.csv"))


In [4]:
print(train_files)

[PosixPath('/root/.keras/datasets/jsb_chorales/train/chorale_000.csv'), PosixPath('/root/.keras/datasets/jsb_chorales/train/chorale_001.csv'), PosixPath('/root/.keras/datasets/jsb_chorales/train/chorale_002.csv'), PosixPath('/root/.keras/datasets/jsb_chorales/train/chorale_003.csv'), PosixPath('/root/.keras/datasets/jsb_chorales/train/chorale_004.csv'), PosixPath('/root/.keras/datasets/jsb_chorales/train/chorale_005.csv'), PosixPath('/root/.keras/datasets/jsb_chorales/train/chorale_006.csv'), PosixPath('/root/.keras/datasets/jsb_chorales/train/chorale_007.csv'), PosixPath('/root/.keras/datasets/jsb_chorales/train/chorale_008.csv'), PosixPath('/root/.keras/datasets/jsb_chorales/train/chorale_009.csv'), PosixPath('/root/.keras/datasets/jsb_chorales/train/chorale_010.csv'), PosixPath('/root/.keras/datasets/jsb_chorales/train/chorale_011.csv'), PosixPath('/root/.keras/datasets/jsb_chorales/train/chorale_012.csv'), PosixPath('/root/.keras/datasets/jsb_chorales/train/chorale_013.csv'), Posix

In [5]:
import pandas as pd

def load_chorales(filepaths):
    # Carrega cada arquivo CSV em filepaths usando o pandas e converte o conteúdo para uma lista de listas
    return [pd.read_csv(filepath).values.tolist() for filepath in filepaths]

# Carrega os dados dos arquivos de treinamento usando a função load_chorales
train_chorales = load_chorales(train_files)

# Carrega os dados dos arquivos de validação usando a função load_chorales
valid_chorales = load_chorales(valid_files)

# Carrega os dados dos arquivos de teste usando a função load_chorales
test_chorales = load_chorales(test_files)


In [6]:
train_chorales[0]

[[74, 70, 65, 58],
 [74, 70, 65, 58],
 [74, 70, 65, 58],
 [74, 70, 65, 58],
 [75, 70, 58, 55],
 [75, 70, 58, 55],
 [75, 70, 60, 55],
 [75, 70, 60, 55],
 [77, 69, 62, 50],
 [77, 69, 62, 50],
 [77, 69, 62, 50],
 [77, 69, 62, 50],
 [77, 70, 62, 55],
 [77, 70, 62, 55],
 [77, 69, 62, 55],
 [77, 69, 62, 55],
 [75, 67, 63, 48],
 [75, 67, 63, 48],
 [75, 69, 63, 48],
 [75, 69, 63, 48],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [75, 69, 63, 48],
 [75, 69, 63, 48],
 [75, 67, 63, 48],
 [75, 67, 63, 48],
 [77, 65, 62, 50],
 [77, 65, 62, 50],
 [77, 65, 60, 50],
 [77, 65, 60, 50],
 [74, 67, 58, 55],
 [74, 67, 58, 55],
 [74, 67, 58, 53],
 [74, 67, 58, 53],
 [72, 67, 58, 51],
 [72, 67, 58, 51],
 [72, 67, 58, 51],
 [72, 67, 58, 51],
 [72, 65, 57

Notas de 36 (C1 = C na oitava 1) até 81 (A5 = A na oitava 5) e 0 para
silêncio.

Por exemplo: `[74, 70, 65, 58]` corresponde à: `[D, Bb, F, B]` - Acorde de Bb (Si bemol maior)

In [7]:
# Assegurando as notas mínimas e máximas do conjuntos de dados.
notes = set()  # Cria um conjunto vazio para armazenar as notas

# Percorre os conjuntos de treinamento, validação e teste
for chorales in (train_chorales, valid_chorales, test_chorales):
    for chorale in chorales:  # Percorre cada chorale dentro do conjunto
        for chord in chorale:  # Percorre cada chord dentro do chorale
            notes |= set(chord)  # Adiciona as notas do chord ao conjunto 'notes'

n_notes = len(notes)  # Obtém o número total de notas únicas
min_note = min(notes - {0})  # Encontra a nota mínima, excluindo a nota 0
max_note = max(notes)  # Encontra a nota máxima

# Verifica se as notas mínima e máxima estão corretas, caso contrário, interrompe
# a execução do programa
assert min_note == 36
assert max_note == 81


Neste código, um conjunto vazio chamado `notes` é criado para armazenar as notas encontradas nos dados musicais. Em seguida, um loop é realizado sobre os conjuntos de treinamento, validação e teste. Dentro de cada conjunto, são percorridos os chorales e, em seguida, os chords de cada chorale. As notas de cada chord são adicionadas ao conjunto `notes` usando a operação de união (`|=`).

Após o loop, o número total de notas únicas é obtido através do comprimento do conjunto `notes` armazenado na variável `n_notes`. A nota mínima é encontrada usando a função `min`, excluindo a nota 0 do conjunto `notes - {0}` e armazenada na variável `min_note`. A nota máxima é encontrada usando a função `max` no conjunto `notes` e armazenada na variável `max_note`.

Em seguida, o código verifica se as notas mínima e máxima estão corretas usando as declarações assert. Se as verificações falharem, será lançada uma exceção para indicar que algo está errado. Neste caso, o código verifica se `min_note` é igual a 36 e se `max_note` é igual a 81.

## Ouvindo o Dataset

Vamos escrever algumas funções para ouvir esses chorais escrevendo um sintetizador para traduzir o dataset em audio.

In [8]:
from IPython.display import Audio

In [9]:
def notes_to_frequencies(notes):
    # A frequência dobra ao subir uma oitava; existem 12 semitons em cada oitava;
    # A nota A na oitava 3 é 440 Hz, e ela corresponde ao número 69.
    return 2 ** ((np.array(notes) - 69) / 12) * 440

Esse código define a função `notes_to_frequencies` que converte números de notas musicais em suas respectivas frequências em hertz.

A função recebe como entrada um array ou lista de números de notas, onde cada número representa uma nota musical. O código então realiza o seguinte cálculo para converter cada número de nota em sua frequência correspondente:

* Subtrai 69 de cada número de nota para normalizar a escala. Isso é feito porque o número 69 corresponde à nota A na oitava 3, que possui uma frequência de 440 Hz.

* Divide o resultado anterior por 12 para obter a diferença em semitons em relação à nota A na oitava 3.

* Calcula 2 elevado ao valor obtido no passo anterior. Isso é feito porque a frequência dobra a cada oitava (que tem 12 semitons) ao subir uma oitava.

* Multiplica o resultado do passo anterior por 440, que é a frequência da nota A na oitava 3.

O resultado final é um array ou lista com as frequências correspondentes às notas musicais fornecidas. Essas frequências indicam a quantidade de oscilações por segundo para cada nota, e são usadas para gerar sons musicais com as notas específicas.

In [10]:
def frequencies_to_samples(frequencies, tempo, sample_rate):
    note_duration = 60 / tempo  # o tempo é medido em batidas por minuto
    # Para reduzir o som de clique a cada batida, arredondamos as frequências para tentar
    # obter as amostras próximas de zero no final de cada nota.
    frequencies = np.round(note_duration * frequencies) / note_duration
    n_samples = int(note_duration * sample_rate)
    time = np.linspace(0, note_duration, n_samples)
    sine_waves = np.sin(2 * np.pi * frequencies.reshape(-1, 1) * time)
    # Removendo todas as notas com frequências ≤ 9 Hz (inclui a nota 0 = silêncio)
    sine_waves *= (frequencies > 9.).reshape(-1, 1)
    return sine_waves.reshape(-1)

Esse código define a função `frequencies_to_samples` que converte frequências em amostras de áudio.

A função recebe três argumentos:

* `frequencies`: uma lista ou array contendo as frequências das notas musicais.
* `tempo`: o tempo em batidas por minuto.
* `sample_rate`: a taxa de amostragem, ou seja, o número de amostras de áudio por segundo.
O código realiza as seguintes etapas para converter as frequências em amostras de áudio:

* Calcula a duração de uma nota, dividindo 60 pelo tempo em batidas por minuto. Isso fornece a duração de cada nota em segundos.

* Arredonda as frequências multiplicadas pela duração da nota para tentar obter amostras próximas de zero no final de cada nota. Isso é feito para reduzir o som de clique que pode ocorrer quando há uma mudança abrupta de frequência entre as notas.

* Calcula o número de amostras necessário para representar a duração da nota, multiplicando a duração da nota pelo sample_rate.

* Cria um array chamado time que representa o tempo para cada amostra de áudio. Esse array é criado usando `np.linspace` para gerar valores igualmente espaçados de 0 a duração da nota.

* Calcula formas de onda senoidais (`sine_waves`) multiplicando as frequências pelas amostras de tempo e pelas constantes relacionadas a 2π. Isso cria uma forma de onda para cada frequência.

* Multiplica as formas de onda senoidais pelo resultado da comparação entre as frequências e 9. Essa comparação cria um array booleano indicando quais frequências são maiores que 9 Hz. Isso é feito para remover as notas com frequências muito baixas, incluindo a nota 0, que representa o silêncio.

* Retorna as formas de onda senoidais resultantes como um array unidimensional.

Em resumo, essa função gera amostras de áudio a partir das frequências das notas musicais. As amostras representam formas de onda senoidais para cada nota, com ajustes para evitar cliques e remover notas de frequência muito baixa. Essas amostras podem ser usadas para reproduzir as notas musicais com as frequências correspondentes.

In [11]:
def chords_to_samples(chords, tempo, sample_rate):
    freqs = notes_to_frequencies(chords)
    freqs = np.r_[freqs, freqs[-1:]]  # faz a última nota um pouco mais longa
    merged = np.mean([frequencies_to_samples(melody, tempo, sample_rate)
                     for melody in freqs.T], axis=0)
    n_fade_out_samples = sample_rate * 60 // tempo  # fade out na última nota
    fade_out = np.linspace(1., 0., n_fade_out_samples)**2
    merged[-n_fade_out_samples:] *= fade_out
    return merged

Esse código define a função `chords_to_samples` que converte acordes musicais em amostras de áudio.

A função recebe três argumentos:

* `chords`: uma lista ou array contendo os acordes musicais representados como números de notas.
* `tempo`: o tempo em batidas por minuto.
* `sample_rate`: a taxa de amostragem, ou seja, o número de amostras de áudio por segundo.
O código realiza as seguintes etapas para converter os acordes em amostras de áudio:

* Chama a função `notes_to_frequencies` para converter os números de notas dos acordes em suas respectivas frequências.

* Adiciona a última frequência duplicada na lista de frequências. Isso faz com que a última nota do acorde seja um pouco mais longa em relação às outras.

* Usa uma compreensão de lista para chamar a função `frequencies_to_samples` para cada melodia (conjunto de frequências) na transposta das frequências. A função `frequencies_to_samples` gera amostras de áudio para cada melodia com base no tempo e taxa de amostragem fornecidos.

* Calcula a duração do fade-out na última nota, multiplicando a taxa de amostragem pelo número de segundos em um minuto dividido pelo tempo. Isso determina quantas amostras serão usadas para criar o fade-out.

* Cria um array chamado `fade_out` que contém uma sequência de valores de intensidade decrescentes, variando de 1 a 0. Esses valores são usados para aplicar um efeito de fade-out na última nota.

* Multiplica as amostras da última nota pelo array `fade_out`, fazendo com que a intensidade das amostras diminua gradualmente.

* Calcula a média das amostras de áudio de todas as melodias dos acordes usando `np.mean` ao longo do eixo 0. Isso cria um único conjunto de amostras de áudio que representa todos os acordes.

* Retorna as amostras de áudio resultantes como um array unificado.

Em resumo, essa função converte acordes musicais em amostras de áudio que podem ser reproduzidas para ouvir os acordes em forma de som.

In [12]:
def play_chords(chords, tempo=160, amplitude=0.1, sample_rate=44100, filepath=None):
    samples = amplitude * chords_to_samples(chords, tempo, sample_rate)
    if filepath:
        from scipy.io import wavfile
        samples = (2**15 * samples).astype(np.int16)
        wavfile.write(filepath, sample_rate, samples)
        return display(Audio(filepath))
    else:
        return display(Audio(samples, rate=sample_rate))

Esse código define a função `play_chords` que reproduz acordes musicais.

A função recebe vários argumentos opcionais:

* `chords`: uma lista ou array contendo os acordes musicais representados como números de notas.
* `tempo`: o tempo em batidas por minuto (padrão: 160).
* `amplitude`: a amplitude do som dos acordes (padrão: 0.1).
* `sample_rate`: a taxa de amostragem, ou seja, o número de amostras de áudio por segundo (padrão: 44100).
* `filepath`: o caminho do arquivo WAV para salvar o áudio (opcional).
O código realiza as seguintes etapas:

* Chama a função `chords_to_samples` para converter os acordes em amostras de áudio, com base nos parâmetros fornecidos (tempo, taxa de amostragem).

* Multiplica as amostras de áudio pelo valor da amplitude para controlar o volume do som dos acordes.

* Se um caminho de arquivo (`filepath`) for fornecido, o código salva as amostras de áudio em um arquivo WAV usando a biblioteca `scipy.io.wavfile`.

* Se um caminho de arquivo foi fornecido, a função retorna `display(Audio(filepath))`, que exibe um player de áudio no ambiente em que o código está sendo executado, permitindo a reprodução do áudio a partir do arquivo.

* Se nenhum caminho de arquivo foi fornecido, a função retorna `display(Audio(samples, rate=sample_rate))`, que exibe um player de áudio com base nas amostras de áudio geradas, permitindo a reprodução direta do áudio.

Em resumo, essa função reproduz acordes musicais, convertendo-os em amostras de áudio e permitindo a reprodução do áudio gerado. O áudio pode ser reproduzido diretamente ou salvo em um arquivo WAV, dependendo dos parâmetros fornecidos.

Portanto, em linhas gerais, nessas funções temos as funções fornecem utilidades relacionadas à geração e reprodução de notas e acordes musicais. Cada função possui um comentário explicando sua finalidade. 

* A função `notes_to_frequencies` converte números de notas em suas respectivas frequências em hertz. 

* A função `frequencies_to_samples` gera formas de onda senoidais com base nas frequências, duração das notas e taxa de amostragem especificadas. 

* A função `chords_to_samples` mescla várias melodias de acordes em uma única amostra de áudio. 

* A função `play_chords` reproduz os acordes gerados, podendo salvar o áudio em um arquivo WAV se o caminho do arquivo for fornecido.

Lembrando que esse código faz uso de bibliotecas como `numpy`, `IPython.display` e `scipy.io` para manipulação de áudio e cálculos musicais.

In [13]:
for index in range(5):
    play_chords(train_chorales[index])

Output hidden; open in https://colab.research.google.com to view.

A fim de ser capaz de gerar novos chorais, queremos treinar um modelo que possa prever o próximo acorde dado todos os acordes anteriores. Se tentarmos ingenuamente prever o próximo acorde de uma vez, prevendo todas as 4 notas de uma só vez, corremos o risco de obter notas que não combinam muito bem juntas (a não ser que você seja um conhecedor e especialista em harmonia musical!). 
Assim, é muito melhor e mais simples prever uma nota de cada vez. 

Portanto, precisaremos pré-processar cada coral, transformando cada acorde em um arpejo (ou seja, uma sequência de notas em vez de notas tocadas simultaneamente). 

Assim, cada coral será uma longa sequência de notas (em vez de acordes), e podemos simplesmente treinar um modelo que possa prever a próxima nota dado todas as notas anteriores. Usaremos uma abordagem de sequência para sequência, onde alimentamos uma janela para a rede neural e ela tenta prever essa mesma janela deslocada um passo de tempo para o futuro.

Também iremos ajustar os valores para que eles variem de 0 a 46, onde 0 representa silêncio, e os valores de 1 a 46 representam as notas de 36 (C1 - Dó1) a 81 (A5 - Lá5).

E treinaremos o modelo em janelas de 128 notas (ou seja, 32 acordes).

Como o conjunto de dados cabe na memória, poderíamos pré-processar os corais na RAM usando qualquer código Python que desejarmos.

In [14]:
def create_target(batch):
    X = batch[:, :-1]
    Y = batch[:, 1:]  # prever a próxima nota em cada arpejo, em cada passo
    return X, Y

def preprocess(window):
    window = tf.where(window == 0, window, window - min_note + 1)  # deslocar valores
    return tf.reshape(window, [-1])  # converter para arpejo

def bach_dataset(chorales, batch_size=32, shuffle_buffer_size=None,
                 window_size=32, window_shift=16, cache=True):
    def batch_window(window):
        return window.batch(window_size + 1)

    def to_windows(chorale):
        dataset = tf.data.Dataset.from_tensor_slices(chorale)
        dataset = dataset.window(window_size + 1, window_shift, drop_remainder=True)
        return dataset.flat_map(batch_window)

    chorales = tf.ragged.constant(chorales, ragged_rank=1)
    dataset = tf.data.Dataset.from_tensor_slices(chorales)
    dataset = dataset.flat_map(to_windows).map(preprocess)
    if cache:
        dataset = dataset.cache()
    if shuffle_buffer_size:
        dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(create_target)
    return dataset.prefetch(1)

Neste código, são definidas várias funções para pré-processar e criar um conjunto de dados (dataset) a partir dos chorais de Bach:

* A função `create_target` recebe um lote (batch) de dados e retorna as entradas `X` e os alvos `Y`. A entrada `X` é formada pelos primeiros elementos de cada sequência no lote, e os alvos `Y` são formados pelos elementos seguintes em cada sequência.

* A função `preprocess` recebe uma janela de dados e realiza o pré-processamento necessário. Ela faz um deslocamento nos valores da janela, subtraindo o valor mínimo (`min_note`) e adicionando 1. Em seguida, a janela é convertida em um arpejo (uma sequência linearizada) por meio da função `tf.reshape`.

* A função `bach_dataset` recebe os chorais de Bach e os parâmetros relacionados à criação do conjunto de dados. Ela cria janelas deslizantes a partir dos chorais, utilizando o tamanho da janela e o deslocamento especificados. Os chorais são convertidos em um formato apropriado (`tf.ragged.constant`) e, em seguida, o conjunto de dados é construído por meio de uma sequência de operações. O conjunto de dados resultante é pré-processado com a função `preprocess`, podendo ser armazenado em cache (se o parâmetro cache for True) e embaralhado (se o parâmetro `shuffle_buffer_size` for especificado). Em seguida, o conjunto de dados é agrupado em lotes, mapeado para criar os pares de entrada e alvo com a função `create_target` e, finalmente, pré-carregado em memória com `prefetch`.

Essa função `bach_dataset` é útil para criar um conjunto de dados a partir dos chorais de Bach, a fim de treinar um modelo de aprendizado de máquina capaz de prever as próximas notas musicais com base nas notas anteriores.

## Criação do conjunto de treino, teste e validação

In [15]:
train_set = bach_dataset(train_chorales, shuffle_buffer_size=1000)
valid_set = bach_dataset(valid_chorales)
test_set = bach_dataset(test_chorales)

* `train_set = bach_dataset(train_chorales, shuffle_buffer_size=1000)`: Cria um conjunto de dados de treinamento utilizando a função `bach_dataset`. É passado como argumento `train_chorales`, que representa os corais de treinamento. Além disso, é especificado `shuffle_buffer_size=1000`, que define o tamanho do buffer de embaralhamento utilizado durante o treinamento. Isso implica que os dados de treinamento serão embaralhados aleatoriamente durante o treinamento do modelo.

* `valid_set = bach_dataset(valid_chorales)`: Cria um conjunto de dados de validação utilizando a função `bach_dataset`. É passado como argumento `valid_chorales`, que representa os corais de validação. Nesse caso, não é especificado um tamanho de buffer de embaralhamento, portanto, não haverá embaralhamento dos dados de validação durante o treinamento.

* `test_set = bach_dataset(test_chorales)`: Cria um conjunto de dados de teste utilizando a função bach_dataset. É passado como argumento test_chorales, que representa os corais de teste. Da mesma forma que o conjunto de validação, não é especificado um tamanho de buffer de embaralhamento para os dados de teste.

## Criando o modelo.

* Poderíamos alimentar os valores das notas diretamente para o modelo, como floats, mas isso provavelmente não resultaria em bons resultados. De fato, as relações entre as notas não são tão simples assim: por exemplo, se substituirmos um Dó3 por um Dó4, a melodia ainda soará bem, mesmo que essas notas estejam separadas por 12 semitons (ou seja, uma oitava). Por outro lado, se substituirmos um Dó3 por um Dó#3, é muito provável que o acorde soe horrível, apesar dessas notas estarem próximas uma da outra. Portanto, vamos usar uma camada de Embedding para converter cada nota em uma representação vetorial pequena. Vamos usar embeddings de 5 dimensões, então a saída dessa primeira camada terá formato [tamanho_lote, tamanho_janela, 5].
* Em seguida, vamos alimentar esses dados em uma pequena rede neural do tipo WaveNet, composta por uma sequência de 4 camadas Conv1D com taxas de dilatação em duplicação. Vamos intercalar essas camadas com camadas de BatchNormalization para uma convergência mais rápida e melhor.
* Em seguida, uma camada LSTM para tentar capturar padrões de longo prazo.
* E, por fim, uma camada Dense para produzir as probabilidades finais das notas. Ela vai prever uma probabilidade para cada choral no lote, para cada passo de tempo, e para cada possível nota (incluindo o silêncio). Portanto, o formato de saída será [tamanho_lote, tamanho_janela, 47].

In [35]:
n_embedding_dims = 5
# Define o número de dimensões de incorporação para a camada de incorporação

model = keras.models.Sequential([
    keras.layers.Embedding(input_dim=n_notes, output_dim=n_embedding_dims,
                           input_shape=[None]),
    # Adiciona uma camada de incorporação que mapeia índices de notas para vetores de tamanho n_embedding_dims.
    # O número de índices possíveis é definido por n_notes.
    # A dimensão de entrada da camada é [None], indicando uma sequência de entrada de comprimento variável.

    keras.layers.Conv1D(32, kernel_size=2, padding="causal", activation="relu"),
    keras.layers.BatchNormalization(),
    # Adiciona uma camada de convolução 1D com 32 filtros e tamanho de kernel 2.
    # A opção padding="causal" aplica preenchimento apenas no lado esquerdo da sequência, preservando a ordem temporal.
    # A ativação ReLU é usada e, em seguida, uma camada de normalização em lote (Batch Normalization) é adicionada.

    keras.layers.Conv1D(48, kernel_size=2, padding="causal", activation="relu", dilation_rate=2),
    keras.layers.BatchNormalization(),
    # Adiciona outra camada de convolução 1D com 48 filtros, tamanho de kernel 2 e taxa de dilatação 2.
    # A taxa de dilatação permite capturar dependências mais distantes na sequência.
    # A ativação ReLU é usada e, em seguida, uma camada de normalização em lote é adicionada.

    keras.layers.Conv1D(64, kernel_size=2, padding="causal", activation="relu", dilation_rate=4),
    keras.layers.BatchNormalization(),
    # Adiciona mais uma camada de convolução 1D com 64 filtros, tamanho de kernel 2 e taxa de dilatação 4.
    # Utiliza ativação ReLU e normalização em lote.

    keras.layers.Conv1D(96, kernel_size=2, padding="causal", activation="relu", dilation_rate=8),
    keras.layers.BatchNormalization(),
    # Adiciona uma última camada de convolução 1D com 96 filtros, tamanho de kernel 2 e taxa de dilatação 8.
    # Utiliza ativação ReLU e normalização em lote.

    keras.layers.LSTM(256, return_sequences=True),
    # Adiciona uma camada de célula LSTM com 256 unidades.
    # A opção return_sequences=True faz com que a camada retorne a sequência completa em vez do último estado oculto.

    keras.layers.Dense(n_notes, activation="softmax")
    # Adiciona uma camada densa com o mesmo número de unidades que o número de notas possíveis.
    # A ativação softmax é usada para obter probabilidades de distribuição para cada nota.
])

model.summary()
# Exibe um resumo do modelo, mostrando a arquitetura da rede neural com o número de parâmetros e dimensões em cada camada.


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, None, 5)           235       
                                                                 
 conv1d_30 (Conv1D)          (None, None, 32)          352       
                                                                 
 batch_normalization_25 (Bat  (None, None, 32)         128       
 chNormalization)                                                
                                                                 
 conv1d_31 (Conv1D)          (None, None, 48)          3120      
                                                                 
 batch_normalization_26 (Bat  (None, None, 48)         192       
 chNormalization)                                                
                                                                 
 conv1d_32 (Conv1D)          (None, None, 64)         

**Obs:**

Uma camada de incorporação (Embedding Layer) é uma camada em uma rede neural que mapeia índices inteiros para vetores de valores contínuos de dimensão menor. Ela é frequentemente usada para representar dados categóricos ou discretos, como palavras em um texto ou notas musicais.

A camada de incorporação é inicializada com uma matriz de pesos que é aprendida durante o treinamento do modelo. Cada índice inteiro é mapeado para um vetor de números reais de tamanho especificado, conhecido como dimensão de incorporação. Esses vetores de incorporação capturam a semântica e as relações entre os índices, permitindo que o modelo aprenda representações densas e distribuídas dos dados.

Por exemplo, em um problema de processamento de texto, onde as palavras são representadas por índices inteiros, uma camada de incorporação pode mapear cada índice de palavra para um vetor de incorporação que representa a semântica da palavra. Palavras semanticamente relacionadas tendem a ter vetores de incorporação próximos no espaço de incorporação.

Essa representação de baixa dimensão e densa fornecida pela camada de incorporação ajuda a reduzir a dimensionalidade dos dados e capturar características importantes dos índices de entrada. Além disso, ela permite que o modelo generalize para índices não vistos durante o treinamento, aprendendo uma representação densa para palavras ou outras categorias discretas.

No contexto do código fornecido, a camada de incorporação é usada para mapear índices de notas musicais para vetores de incorporação de tamanho `n_embedding_dims`. Isso permite que o modelo aprenda representações de notas musicais em um espaço de incorporação de dimensão reduzida, capturando características relevantes para o problema de previsão de notas musicais.

## Treinando o modelo

In [36]:
optimizer = keras.optimizers.Nadam(learning_rate=1e-3)
# Cria um otimizador Nadam com uma taxa de aprendizagem de 1e-3.
# O Nadam é um otimizador baseado no algoritmo Adam com Nesterov momentum.

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
# Compila o modelo com a função de perda "sparse_categorical_crossentropy",
# que é adequada para problemas de classificação com várias classes.
# Utiliza o otimizador definido anteriormente e define a métrica de avaliação como "accuracy" (acurácia),
# que mede a precisão do modelo durante o treinamento.

model.fit(train_set, epochs=20, validation_data=valid_set)
# Realiza o treinamento do modelo com o conjunto de treinamento `train_set` por 20 épocas.
# O conjunto de validação `valid_set` é utilizado para avaliar o desempenho do modelo após cada época de treinamento.
# Durante o treinamento, o modelo tenta minimizar a função de perda especificada usando o otimizador definido.


Epoch 1/20
98/98 [==============================] - 19s 41ms/step - loss: 1.8979 - accuracy: 0.5240 - val_loss: 4.2566 - val_accuracy: 0.0421
Epoch 2/20
98/98 [==============================] - 2s 24ms/step - loss: 0.8807 - accuracy: 0.7675 - val_loss: 4.3852 - val_accuracy: 0.0303
Epoch 3/20
98/98 [==============================] - 2s 24ms/step - loss: 0.7328 - accuracy: 0.7965 - val_loss: 3.8434 - val_accuracy: 0.0906
Epoch 4/20
98/98 [==============================] - 2s 20ms/step - loss: 0.6570 - accuracy: 0.8117 - val_loss: 3.0815 - val_accuracy: 0.1855
Epoch 5/20
98/98 [==============================] - 2s 18ms/step - loss: 0.6055 - accuracy: 0.8232 - val_loss: 1.8434 - val_accuracy: 0.4390
Epoch 6/20
98/98 [==============================] - 2s 23ms/step - loss: 0.5640 - accuracy: 0.8330 - val_loss: 0.9228 - val_accuracy: 0.7287
Epoch 7/20
98/98 [==============================] - 2s 19ms/step - loss: 0.5303 - accuracy: 0.8409 - val_loss: 0.6997 - val_accuracy: 0.7987
Epoch 8/20
9

## Salvando o modelo

In [22]:
model.save("my_bach_model.h5")
model.evaluate(test_set)

34/34 [==============================] - 1s 13ms/step - loss: 0.6426 - accuracy: 0.8196


[0.6426379084587097, 0.819591224193573]

* `model.save("my_bach_model.h5")`: Salva o modelo treinado em um arquivo com o nome "my_bach_model.h5". O formato ".h5" é comumente usado para salvar modelos treinados em frameworks como o TensorFlow e o Keras. O modelo é salvo em disco para que possa ser carregado e reutilizado posteriormente sem precisar treiná-lo novamente.

* `model.evaluate(test_set)`: Avalia o desempenho do modelo no conjunto de teste. O `test_set` refere-se ao conjunto de dados de teste que foi separado anteriormente. A função `evaluate` executa a inferência do modelo nos dados de teste e calcula as métricas de desempenho, como a precisão, a perda ou outras métricas relevantes, dependendo do problema. Isso permite avaliar o quão bem o modelo generaliza para novos dados e medir a sua performance.

**Observação:** Não há uma necessidade real de um conjunto de teste neste exercício, uma vez que faremos a avaliação final apenas ouvindo a música produzida pelo modelo. Portanto, se desejar, você pode adicionar o conjunto de teste ao conjunto de treinamento e treinar o modelo novamente, esperando obter um modelo um pouco melhor.

Agora vamos escrever uma função que irá gerar um novo coral. Daremos a ela alguns acordes iniciais, ela os converterá em arpejos (o formato esperado pelo modelo) e usará o modelo para prever a próxima nota, depois a próxima, e assim por diante. No final, ela agrupará as notas em conjuntos de 4 para criar acordes novamente e retornará o coral resultante.

Aviso: `model.predict_classes(X)` está obsoleto. Ele foi substituído por `np.argmax(model.predict(X), axis=-1)`.

## Gerando um novo coral

In [23]:
def generate_chorale(model, seed_chords, length):
    arpegio = preprocess(tf.constant(seed_chords, dtype=tf.int64))  # Pré-processa os acordes iniciais para obter o arpejo
    arpegio = tf.reshape(arpegio, [1, -1])  # Redimensiona o arpejo para ter formato [1, comprimento]
    for chord in range(length):  # Loop para gerar acordes adicionais
        for note in range(4):  # Loop para gerar cada nota do acorde
            # Próxima nota com base nas previsões do modelo
            next_note = np.argmax(model.predict(arpegio), axis=-1)[:1, -1:]  
            arpegio = tf.concat([arpegio, next_note], axis=1)  # Concatena a próxima nota ao arpejo
    arpegio = tf.where(arpegio == 0, arpegio, arpegio + min_note - 1)  # Ajusta os valores do arpejo
    return tf.reshape(arpegio, shape=[-1, 4])  # Redimensiona o arpejo para ter formato de acordes

Neste código, a função `generate_chorale` recebe um modelo treinado, uma lista de acordes iniciais (`seed_chords`) e o comprimento desejado para o coral resultante (`length`).

* Primeiro, os acordes iniciais são pré-processados para obter o arpejo correspondente usando a função preprocess. Os acordes iniciais são convertidos em arpejo, que é o formato esperado pelo modelo.

* Em seguida, o arpejo é redimensionado para ter formato [1, comprimento], para representar um único coral com o comprimento especificado.

* Em um loop externo, o código itera sobre cada acorde a ser gerado no coral resultante.

* Dentro desse loop externo, há um loop interno que itera sobre cada nota do acorde.

* A próxima nota é gerada com base nas previsões do modelo. A linha `next_note = np.argmax(model.predict(arpegio), axis=-1)[:1, -1:]` usa o modelo para prever a próxima nota do arpejo e seleciona apenas a última nota gerada.

* A próxima nota é concatenada ao arpejo usando `tf.concat`, expandindo o comprimento do arpejo.

* Após gerar todas as notas do acorde, o código ajusta os valores do arpejo usando `tf.where`, adicionando `min_note - 1` aos valores não nulos, para retornar as notas originais do coral.

* Por fim, o arpejo é redimensionado novamente para ter o formato de acordes [número_de_acordes, 4], onde cada acorde contém 4 notas.

Essa função gera um coral completo com base nos acordes iniciais fornecidos e usando o modelo para prever as próximas notas do arpejo.

Para testar essa função, precisamos de alguns acordes iniciais. Vamos usar os primeiros 16 acordes de um dos chorais de teste (na verdade, são apenas 4 acordes diferentes, cada um tocado 4 vezes):

In [24]:
seed_chords = test_chorales[2][:16]
play_chords(seed_chords, amplitude=0.2)

Vamos pedir à função para gerar mais 56 acordes, totalizando 64 acordes, ou seja, 16 compassos (assumindo 4 acordes por compasso, ou seja, uma assinatura de 4/4):

In [37]:
new_chorale = generate_chorale(model, seed_chords, 112)
play_chords(new_chorale)

Output hidden; open in https://colab.research.google.com to view.

Essa abordagem costuma ser muito conservadora. De fato, o modelo não assume riscos, sempre escolhendo a nota com a pontuação mais alta. E como repetir a nota anterior geralmente soa bem o suficiente, é a opção menos arriscada. 

Assim, o algoritmo tenderá a fazer as notas durarem cada vez mais. Isso acaba se tornando bastante monótono. Além disso, se você executar o modelo várias vezes, ele sempre gerará a mesma melodia.

Portanto, ao invés de sempre escolher a nota com a pontuação mais alta, iremos escolher a próxima nota aleatoriamente, de acordo com as probabilidades previstas. 

Por exemplo, se o modelo prevê um C3 com 75% de probabilidade e um G3 com 25% de probabilidade, escolheremos uma dessas duas notas aleatoriamente, seguindo essas probabilidades. 

Também adicionaremos um parâmetro de temperatura que controlará o quão "quente" (ousado) queremos que o sistema pareça. Uma temperatura alta fará com que as probabilidades previstas fiquem mais próximas umas das outras, reduzindo a probabilidade das notas mais prováveis e aumentando a probabilidade das menos prováveis.

In [39]:
def generate_chorale_v2(model, seed_chords, length, temperature=1):
    arpegio = preprocess(tf.constant(seed_chords, dtype=tf.int64))  # Pré-processa os acordes iniciais para obter o arpejo
    arpegio = tf.reshape(arpegio, [1, -1])  # Redimensiona o arpejo para ter formato [1, comprimento]
    for chord in range(length):  # Loop para gerar acordes adicionais
        for note in range(4):  # Loop para gerar cada nota do acorde
            next_note_probas = model.predict(arpegio)[0, -1:]  # Probabilidades previstas para a próxima nota
            rescaled_logits = tf.math.log(next_note_probas) / temperature  # Ajusta as probabilidades de acordo com a temperatura
            next_note = tf.random.categorical(rescaled_logits, num_samples=1)  # Escolhe a próxima nota aleatoriamente de acordo com as probabilidades ajustadas
            arpegio = tf.concat([arpegio, next_note], axis=1)  # Concatena a próxima nota ao arpejo
    arpegio = tf.where(arpegio == 0, arpegio, arpegio + min_note - 1)  # Ajusta os valores do arpejo
    return tf.reshape(arpegio, shape=[-1, 4])  # Redimensiona o arpejo para ter formato de acordes


Neste código, a função `generate_chorale_v2` é uma versão modificada da função anterior que gera um coral com base em acordes iniciais usando o modelo treinado. As modificações introduzem a aleatoriedade nas escolhas das notas de acordo com as probabilidades previstas e a temperatura especificada.

* Os acordes iniciais são pré-processados e convertidos em arpejo usando a função `preprocess`.

* Em seguida, o arpejo é redimensionado para ter formato [1, comprimento].

* O código entra em um loop externo para gerar acordes adicionais.

* Dentro desse loop externo, há um loop interno que itera sobre cada nota do acorde.

* As probabilidades previstas para a próxima nota são obtidas usando `model.predict(arpegio)[0, -1:]`. Essas probabilidades são um vetor que representa a distribuição de probabilidade para cada possível nota.

* Os logitos (log-odds) são recalibrados usando a temperatura especificada. Isso é feito aplicando o logaritmo às probabilidades e dividindo pelo valor da temperatura. A temperatura controla a "ousadia" do sistema, pois afeta a dispersão das probabilidades.

* A próxima nota é escolhida aleatoriamente com base nas probabilidades recalibradas usando `tf.random.categorical`. O número de amostras é definido como 1, para escolher apenas uma nota.

* A próxima nota é concatenada ao arpejo usando `tf.concat`, expandindo o comprimento do arpejo.

* Após gerar todas as notas do acorde, o código ajusta os valores do arpejo usando `tf.where`, adicionando `min_note - 1` aos valores não nulos, para retornar as notas originais do coral.

* Por fim, o arpejo é redimensionado novamente para ter o formato de acordes [número_de_acordes, 4], onde cada acorde contém 4 notas.

Essa função gera um coral com base nos acordes iniciais fornecidos, usando o modelo treinado com a introdução da aleatoriedade nas escolhas das notas de acordo com as probabilidades previstas e a temperatura especificada.

## Criando chorais com diferentes nuaces sonoras

Vamos gerar 3 corais usando essa nova função: um frio, um médio e um quente. 

Mude as sementes, comprimentos, temperaturas e veja os resultsos.

O código salva cada coral em um arquivo separado. Você pode executar essas células quantas vezes quiser!

### Nível conservador

In [40]:
new_chorale_v2_cold = generate_chorale_v2(model, seed_chords, 56, temperature=0.5)
play_chords(new_chorale_v2_cold, filepath="bach_cold.wav")

Output hidden; open in https://colab.research.google.com to view.

### Nível moderado

In [41]:
new_chorale_v2_medium = generate_chorale_v2(model, seed_chords, 56, temperature=1.0)
play_chords(new_chorale_v2_medium, filepath="bach_medium.wav")

Output hidden; open in https://colab.research.google.com to view.

### Nível hardcore

In [43]:
new_chorale_v2_hot = generate_chorale_v2(model, seed_chords, 112, temperature=3)
play_chords(new_chorale_v2_hot, filepath="bach_hot.wav")

Output hidden; open in https://colab.research.google.com to view.

### Coral original

In [ ]:
play_chords(test_chorales[2][:64], filepath="bach_test_4.wav")

## Exercício:

Faça uma busca sobre os hiperparâmetros que podem ser manipulados nesse modelo e tente otimizá-lo. 

Por exemplo, você poderia tentar remover a camada LSTM e substituí-la por camadas Conv1D. Você também poderia experimentar com o número de camadas, a taxa de aprendizado, o otimizador, e assim por diante.

Pesquise e verifique a possibilidade de implementar os seguintes exemplos:

1. Ajustar os hiperparâmetros;
2. Explorar diferentes arquiteturas;
3. Regularização;
4. Ajustar a taxa de aprendizagem;
5. Utilizar regularização de Batch Normalization;
6. Explorar técnicas de ajuste dinâmico da taxa de aprendizagem;
7. Utilizar regularização L2;
8. Aumentar o tamanho do batch;
9. Utilizar uma função de perda ponderada;
10. Utilizar Early Stopping;
11. Explorar técnicas de regularização mais avançadas;

Lembrando que nem todas as técnicas podem ser aplicados ou modificam os resultados obtidos de forma substancial. É necessário avaliar cada uma dessas mudanças.



### Códigos para referência

In [ ]:
# 1. Ajustar os hiperparâmetros;

# Ajustar o número de unidades LSTM para 128
model = keras.models.Sequential([
    ...
    keras.layers.LSTM(128, return_sequences=True),
    ...
])


In [ ]:
# 2. Explorar diferentes arquiteturas;

# Adicionar uma camada de pooling após as camadas convolucionais
model = keras.models.Sequential([
    ...
    keras.layers.Conv1D(32, kernel_size=2, padding="causal", activation="relu"),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Conv1D(48, kernel_size=2, padding="causal", activation="relu", dilation_rate=2),
    keras.layers.MaxPooling1D(pool_size=2),
    ...
])


In [ ]:
# 3. Regularização;

# Adicionar dropout após a camada LSTM
model = keras.models.Sequential([
    ...
    keras.layers.LSTM(256, return_sequences=True),
    keras.layers.Dropout(0.2),
    ...
])


In [ ]:
# 4. Ajustar a taxa de aprendizagem;

# Utilizar o otimizador Adam com uma taxa de aprendizagem menor
optimizer = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])


In [ ]:
# 5. Utilizar regularização de Batch Normalization:

# Certifique-se de que a opção 'training' esteja corretamente definida
keras.layers.BatchNormalization(training=True)


In [ ]:
# 6. Explorar técnicas de ajuste dinâmico da taxa de aprendizagem:

# Utilizar um Learning Rate Schedule para reduzir a taxa de aprendizagem ao longo do tempo
def lr_schedule(epoch):
    if epoch < 10:
        return 0.001
    else:
        return 0.0001

lr_callback = keras.callbacks.LearningRateScheduler(lr_schedule)

model.fit(train_set, epochs=20, callbacks=[lr_callback])


In [ ]:
# 7. Utilizar regularização L2

# Adicionar regularização L2 nas camadas densas
model = keras.models.Sequential([
    ...
    keras.layers.Dense(128, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(n_notes, activation="softmax", kernel_regularizer=keras.regularizers.l2(0.01))
    ...
])


In [ ]:
# 8. Aumentar o tamanho do batch

# Aumentar o tamanho do batch para acelerar o treinamento
train_set = bach_dataset(train_chorales, batch_size=64)


In [ ]:
# 9. Utilizar uma função de perda ponderada

# Utilizar uma função de perda ponderada para lidar com desequilíbrio de classes
weights = compute_class_weights(train_labels)  # Calcular os pesos das classes

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"], class_weight=weights)


In [ ]:
# 10. Utilizar Early Stopping

# Utilizar Early Stopping para interromper o treinamento prematuramente se a perda de validação não melhorar
early_stopping = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

model.fit(train_set, epochs=20, validation_data=valid_set, callbacks=[early_stopping])


In [ ]:
# 11. Explorar técnicas de regularização mais avançadas

# Utilizar Dropout combinado com regularização L1
model = keras.models.Sequential([
    ...
    keras.layers.Dense(128, activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=0.01, l2=0.01)),
    keras.layers.Dropout(0.2),
    ...
])
